In [2]:
import vaex as vx
import pandas as pd
import numpy as np
from tqdm import tqdm
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [2]:
def hash_dict_to_df(dictionary, name):
    df = pd.Series(dictionary,index=dictionary.keys())
    df = df.to_frame().reset_index().rename(columns={0:name})
    return df

# Data Preparartion

In [36]:
train_raw = vx.from_csv("pricing_hackathon_checks_train.csv", header=None)

In [4]:
test_raw = vx.from_csv("pricing_hackathon_checks_test.csv", header=None)

In [3]:
hierarchy = pd.read_csv("pricing_hackathon_hierarchy.csv", header=None)
shops = pd.read_csv("pricing_hackathon_shops.csv", header=None)

In [5]:
hierarchy.rename(columns={0:"sku", 1:"class", 2:"group", 3:"category", 4:"subcategory"}, inplace=True)

In [125]:
sku_dict = dict(zip(hierarchy.sku.values, range(hierarchy['sku'].nunique())))
sku = hash_dict_to_df(sku_dict, "sku")
sku.to_csv("sku.csv", index=None)

,index,sku
0,16b4bff70d409ddb77c8107a528cf94b,0
1,fc6867c8b4ce831c16fa7961958270d4,1
2,1b3353cb266de9c2bcc2fc5b911973f0,2
3,a3d57e327b84fb971e09b2b33c890538,3
4,1c71460770f458c06a29ed650c0974cc,4
...,...,...
123367,2171b026186b6d76891905f2d620de6b,123367
123368,39c9ec2433b27c40cb2c52c1a0d6ed1d,123368
123369,44682cad71db14162d0f3a23ea0b7b05,123369
123370,aeec3b99d48e90e84943545d91f93566,123370


In [6]:
shops.rename(columns={0:"division_id", 1:"region_id", 2:"city_id", 3:"shop_id", 4:"type_loc_id", 
                      5:"type_size_id", 6:"type_format_id", 7:"type_wealth_id", 8:"is_store",
                      9:"is_active", 10:"is_ex_billa"}, inplace=True)

In [130]:
shops_dict = dict(zip(shops.shop_id.values, range(shops['shop_id'].nunique())))
shop_id = hash_dict_to_df(shops_dict, "shop_id")
shop_id.to_csv("shop_id.csv", index=None)

## Replacing hashes with int

In [7]:
column_train_names={"0":"client_id", "1":"day", "2":"shop_id", "3":"check_id", "4":"time", 
              "5":"sku", "6":"promo_id", "7":"check_pos", "8":"num_sales", "9":"supplier_price",
              "10":"selling_price", "11":"discount", "12":"region_name"}

column_test_names={"0":"client_id", "1":"day", "2":"shop_id", "3":"check_id", "4":"time", 
              "5":"sku", "6":"promo_id", "7":"supplier_price",
              "8":"selling_price", "9":"discount", "10":"region_name"}

In [8]:
for name in column_train_names.keys():
    train_raw.rename(name, column_train_names[name])

for name in column_test_names.keys():
    test_raw.rename(name, column_test_names[name])

NameError: name 'train_raw' is not defined

In [38]:
train_raw.drop("region_name", inplace=True)
test_raw.drop("region_name", inplace=True)

#,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,discount
0,644474c3d41185fa4c4f427095954f06,20191017,d2ac32e14d651b9ed03f26f845a11597,f0478c7f392532b5b9843e144a31cb78,14:35:26,98f463552b07d49686c8be996251c778,7215ee9c7d9dc229d2921a40e899ec5f,4,1.0,24.94,48.09,2.6
1,644474c3d41185fa4c4f427095954f06,20191017,d2ac32e14d651b9ed03f26f845a11597,f0478c7f392532b5b9843e144a31cb78,14:35:26,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,1,1.0,1.14,3.79,0.2
2,644474c3d41185fa4c4f427095954f06,20191017,d2ac32e14d651b9ed03f26f845a11597,f0478c7f392532b5b9843e144a31cb78,14:35:26,16b4bff70d409ddb77c8107a528cf94b,7215ee9c7d9dc229d2921a40e899ec5f,8,1.0,85.18,198.18,10.61
3,644474c3d41185fa4c4f427095954f06,20191017,d2ac32e14d651b9ed03f26f845a11597,f0478c7f392532b5b9843e144a31cb78,14:35:26,b0fc4f923bae05bf1276315a8efae07c,7215ee9c7d9dc229d2921a40e899ec5f,3,1.0,29.17,99.99,5.3
4,644474c3d41185fa4c4f427095954f06,20191017,d2ac32e14d651b9ed03f26f845a11597,f0478c7f392532b5b9843e144a31cb78,14:35:26,f0c13fc65a011d296e01626c0c2649d0,e0ceb21aca7b78a5ea6418b0c28e9c95,5,1.476,157.66,174.79,58.26
...,...,...,...,...,...,...,...,...,...,...,...,...
"21,701,352",70067d1dce4f981c6985e15b9aa72a8a,20201114,087c8abfaee44ebbf0c2871976a2ab18,fc87375a3b44f61d077e63bce5e4790a,11:43:56,12919109e871ab98e3bf57ff71f1bc5b,fff5050c55970cb6ae38750cc947a33e,13,1.0,58.05,79.99,30.6
"21,701,353",70067d1dce4f981c6985e15b9aa72a8a,20201114,087c8abfaee44ebbf0c2871976a2ab18,fc87375a3b44f61d077e63bce5e4790a,11:43:56,34b2b0706163451a09a18a71998231ff,369d0d5afe646c011f39484143c7a679,17,1.0,18.02,38.99,7.4
"21,701,354",70067d1dce4f981c6985e15b9aa72a8a,20201114,087c8abfaee44ebbf0c2871976a2ab18,fc87375a3b44f61d077e63bce5e4790a,11:43:56,52db5409c318639598c084ff02b90c18,4f4400004707a9cd38b9c0964d4f6f2b,9,1.0,77.36,119.98,27.41
"21,701,355",70067d1dce4f981c6985e15b9aa72a8a,20201114,087c8abfaee44ebbf0c2871976a2ab18,21ef842bba62869663333693a5f7980c,13:09:19,103e025e3ab812402ee283cfa924348d,6561c4625123c9ca405837b753931345,6,2.0,13.42,0.02,103.16


In [16]:
sku = pd.read_csv("sku.csv")
shop_id = pd.read_csv("shop_id.csv")

In [21]:
sku = sku.set_index('index').to_dict()['sku']
shop_id = shop_id.set_index('index').to_dict()['shop_id']
len(sku), len(shop_id)

(123372, 60)

In [155]:
len(sku), len(shop_id)

(123372, 60)

### Train

In [41]:
#there are 29 skus that are in train but not hierarchy
missing_skus = ['93c7dea84a6b49e2f557a8465c9b10ed', '90a77ce6cae10a339d8d9306e101bc07', '6fd67fb9a6ba0c9e20faa6ab38b43a2a', 'acd66272cf76f932b100b7beca02d18b', '79dd1221d5192b05f58634938bfb928f', '826f20f4d081f2acfe898b907f7e6fdd', 
                '2e8a9c5daae283dd41ac47592f597416', '605c01c65f46ebfc79a0471a06079416', 'f48b2e8c06c6871b247cce89743fd5d5', 'eb74ec52c66ad2860e9b577b1929486d', 'd6a26a60b780a76e3275b02436711700', 'ad1afc08c5f185aead8e10ab805ab3b6', 
                'e596ba3f40949c4b3b7016c4aa9bc028', '05a8c99850d38f831807bd360248be80', 'aef0cc340473c0bec62ef6ae03a6e023', '33fc9ba244f8c64c8387b761ba17ebec', '2dd11001ad097ea96cea414d3f5c8dbf', 'ae6d2c93c4142dc19da51472773423ce', '08091a25cb5fc46b2c9d1a28f5d37d25', 'df68cd0285b098963b50754ba01ddb1f', 'ff923ad4664a24019941ed2e74bb5d7f', '891990aad777e73fe474b806e0002628', '2ea0a95579880cc211f6d6c856f1dd2d', 'bd96f30e5ae2866088279e2fbfefa07a', 'dd2f316343913c91fb2b9d1df1a92471', 'd1adc5dd424576449e1930b5d0af11cc', '64d98a5947885795cd0775ea0f432e5b', '65033bec6580b363ccd4934de0f9582d', '57c8fa3a567c5c683be3057631618962']

train_raw = train_raw[~train_raw['sku'].isin(missing_skus)]

In [42]:
train_raw['shop_id'] = train_raw.shop_id.map(shop_id)
train_raw['sku'] = train_raw.sku.map(sku)

In [43]:
train_raw.drop('client_id', inplace=True)

#,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,discount
0,20191017,55,f0478c7f392532b5b9843e144a31cb78,14:35:26,57189,7215ee9c7d9dc229d2921a40e899ec5f,4,1.0,24.94,48.09,2.6
1,20191017,55,f0478c7f392532b5b9843e144a31cb78,14:35:26,79182,7215ee9c7d9dc229d2921a40e899ec5f,1,1.0,1.14,3.79,0.2
2,20191017,55,f0478c7f392532b5b9843e144a31cb78,14:35:26,0,7215ee9c7d9dc229d2921a40e899ec5f,8,1.0,85.18,198.18,10.61
3,20191017,55,f0478c7f392532b5b9843e144a31cb78,14:35:26,71883,7215ee9c7d9dc229d2921a40e899ec5f,3,1.0,29.17,99.99,5.3
4,20191017,55,f0478c7f392532b5b9843e144a31cb78,14:35:26,26392,e0ceb21aca7b78a5ea6418b0c28e9c95,5,1.476,157.66,174.79,58.26
...,...,...,...,...,...,...,...,...,...,...,...
"21,692,955",20201114,16,fc87375a3b44f61d077e63bce5e4790a,11:43:56,55,fff5050c55970cb6ae38750cc947a33e,13,1.0,58.05,79.99,30.6
"21,692,956",20201114,16,fc87375a3b44f61d077e63bce5e4790a,11:43:56,62011,369d0d5afe646c011f39484143c7a679,17,1.0,18.02,38.99,7.4
"21,692,957",20201114,16,fc87375a3b44f61d077e63bce5e4790a,11:43:56,118935,4f4400004707a9cd38b9c0964d4f6f2b,9,1.0,77.36,119.98,27.41
"21,692,958",20201114,16,21ef842bba62869663333693a5f7980c,13:09:19,40297,6561c4625123c9ca405837b753931345,6,2.0,13.42,0.02,103.16


In [44]:
check_id_dict = dict(zip(train_raw.check_id.values, range(1, len(train_raw)+1)))
check_id_dict_final = dict() 
for old_key in tqdm(check_id_dict.keys()):
    new_key = str(old_key)
    check_id_dict_final[new_key] = check_id_dict[old_key] 

train_raw['check_id'] = train_raw.check_id.map(check_id_dict_final)

In [186]:
promo_id = set(train_raw.promo_id.unique() + test_raw.promo_id.unique())
promo_id_dict = dict(zip(promo_id, range(1, len(promo_id)+1)))
promo_id_dict_final = dict() 
for old_key in tqdm(promo_id_dict.keys()):
    new_key = str(old_key)
    promo_id_dict_final[new_key] = promo_id_dict[old_key]
promo_id = hash_dict_to_df(promo_id_dict_final, "promo_id")
promo_id.to_csv("promo_id.csv", index=None)    

100%|██████████| 275420/275420 [00:00<00:00, 1252906.91it/s]


In [84]:
promo_id = pd.read_csv("promo_id.csv")
promo_id = promo_id.set_index('index').to_dict()['promo_id']

In [47]:
train_raw['promo_id'] = train_raw.promo_id.map(promo_id)

In [52]:
train_raw.export_csv("train_preprocessed.csv", index=False)

In [54]:
train_raw = pd.read_csv("train_preprocessed.csv")
train_raw.head()

,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,discount
0,20191017,55,20990487,14:35:26,57189,51588,4,1.000,24.94,48.09,2.60
1,20191017,55,20990487,14:35:26,79182,51588,1,1.000,1.14,3.79,0.20
2,20191017,55,20990487,14:35:26,0,51588,8,1.000,85.18,198.18,10.61
3,20191017,55,20990487,14:35:26,71883,51588,3,1.000,29.17,99.99,5.30
4,20191017,55,20990487,14:35:26,26392,171211,5,1.476,157.66,174.79,58.26


In [55]:
train_raw["day"] = pd.to_datetime(train_raw["day"], format="%Y%m%d")

In [62]:
train_raw.to_csv("train_preprocessed.csv", index=False)

### Test

In [63]:
test_raw.drop("client_id", inplace=True)

#,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount
0,20211025,6cb9669ff7bbb140212081ccb0f68543,e3bfbc5ed0cc52ebb6a8354dd48134eb,22:25:37,a7f8f533c4f62b8220b6493fef3f5191,9afa17bdb2b819a7d795533045c9c2db,64.84,86.26,28.72
1,20211025,6cb9669ff7bbb140212081ccb0f68543,d8fc54c24b01b8eec0bd790d38fcb3a2,21:25:01,1119d0da7586ec0862aa53521d6a2a62,7dc93b050e25b6990eb7131db59c3a8f,8.8,0.0,51.59
2,20211025,6cb9669ff7bbb140212081ccb0f68543,d8fc54c24b01b8eec0bd790d38fcb3a2,21:25:01,ea88c10b723965c08ab2d54ef53786ac,19121bbfaa43e1f6d552bfe1212ee9a7,384.6,599.0,347.39
3,20211025,6cb9669ff7bbb140212081ccb0f68543,e3bfbc5ed0cc52ebb6a8354dd48134eb,22:25:37,1cbaaeb2212bbbe9d458ca9b81d74db7,07ed162d36d637a07cf5205962b6b73a,28.68,41.27,23.95
4,20211025,6cb9669ff7bbb140212081ccb0f68543,e3bfbc5ed0cc52ebb6a8354dd48134eb,22:25:37,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,1.47,5.49,0.3
...,...,...,...,...,...,...,...,...,...
"2,666,212",20211023,785ee201dc63daae3aecb2c7cbae60d3,09705772612e2a100b5f07c0e57fc524,10:52:58,09ff501c00a9d8b7188a8828075d5737,7215ee9c7d9dc229d2921a40e899ec5f,688.37,986.45,51.92
"2,666,213",20211023,785ee201dc63daae3aecb2c7cbae60d3,09705772612e2a100b5f07c0e57fc524,10:52:58,49d6407eda8665d69f32a107f854435b,025d07e5e7225e0dc6ffd79220a4d199,85.83,99.99,109.5
"2,666,214",20211023,785ee201dc63daae3aecb2c7cbae60d3,09705772612e2a100b5f07c0e57fc524,10:52:58,39f9df15e644860e4a1744e41ce7a6ec,7215ee9c7d9dc229d2921a40e899ec5f,7.78,13.5,0.72
"2,666,215",20211023,785ee201dc63daae3aecb2c7cbae60d3,09705772612e2a100b5f07c0e57fc524,10:52:58,1119d0da7586ec0862aa53521d6a2a62,7dc93b050e25b6990eb7131db59c3a8f,8.8,0.0,51.59


In [66]:
#there are skus that are in test but not hierarchy
missing_skus = ['df68cd0285b098963b50754ba01ddb1f', 'ad1afc08c5f185aead8e10ab805ab3b6', 'f48b2e8c06c6871b247cce89743fd5d5', '605c01c65f46ebfc79a0471a06079416', '65033bec6580b363ccd4934de0f9582d']
test_raw = test_raw[~test_raw['sku'].isin(missing_skus)]


In [85]:
test_raw['shop_id'] = test_raw.shop_id.map(shop_id)
test_raw['sku'] = test_raw.sku.map(sku)
test_raw['promo_id'] = test_raw.promo_id.map(promo_id)

In [87]:
check_id_dict = dict(zip(test_raw.check_id.values, range(1, len(test_raw)+1)))
check_id_dict_final = dict() 
for old_key in tqdm(check_id_dict.keys()):
    new_key = str(old_key)
    check_id_dict_final[new_key] = check_id_dict[old_key] 

test_raw['check_id'] = test_raw.check_id.map(check_id_dict_final)

100%|██████████| 254155/254155 [00:00<00:00, 746029.70it/s]


In [90]:
test_raw.export_csv("test_preprocessed.csv")

In [91]:
test_raw = pd.read_csv("test_preprocessed.csv")
test_raw

,day,shop_id,check_id,time,sku,promo_id,supplier_price,selling_price,discount
0,20211025,23,10,22:25:37,114519,300187,64.84,86.26,28.72
1,20211025,23,11,21:25:01,4405,285514,8.80,0.00,51.59
2,20211025,23,11,21:25:01,99020,296267,384.60,599.00,347.39
3,20211025,23,10,22:25:37,88039,299802,28.68,41.27,23.95
4,20211025,23,10,22:25:37,79182,51588,1.47,5.49,0.30
...,...,...,...,...,...,...,...,...,...
2666079,20211023,42,2666084,10:52:58,70,51588,688.37,986.45,51.92
2666080,20211023,42,2666084,10:52:58,94981,286290,85.83,99.99,109.50
2666081,20211023,42,2666084,10:52:58,96905,51588,7.78,13.50,0.72
2666082,20211023,42,2666084,10:52:58,4405,285514,8.80,0.00,51.59


In [92]:
test_raw["day"] = pd.to_datetime(test_raw["day"], format="%Y%m%d")

In [94]:
test_raw.to_csv("test_preprocessed.csv", index=False)

In [100]:
del check_id_dict
del check_id_dict_final

## Merging additionl tables

In [9]:
train = pd.read_csv("train_preprocessed.csv")

In [10]:
sku = pd.read_csv("sku.csv")
shop_id = pd.read_csv("shop_id.csv")

In [13]:
sku = sku.set_index('index').to_dict()['sku']
shop_id = shop_id.set_index('index').to_dict()['shop_id']

In [14]:
hierarchy['sku'] = hierarchy.sku.map(sku)

In [15]:
hierarchy

,sku,class,group,category,subcategory
0,0,c4ca4238a0b923820dcc509a6f75849b,c4ca4238a0b923820dcc509a6f75849b,7fcc48d22804dbbe9b66b607d51389d4,7b82e428cf5bdd7671ab0f1fa285a767
1,1,c4ca4238a0b923820dcc509a6f75849b,c4ca4238a0b923820dcc509a6f75849b,2a3d6d6cc4b5e77238c1fc1bb6cdd681,ec972afde3cb8ff3e84ab57ffe32f8e4
2,2,c81e728d9d4c2f636f067f89cc14862c,e4da3b7fbbce2345d7772b0674a318d5,19485224d128528da1602ca47383f078,34e14de7a374ed14185fa8e81a5ebb1f
3,3,c81e728d9d4c2f636f067f89cc14862c,e4da3b7fbbce2345d7772b0674a318d5,7bec7e63a493e2d61891b1e4051ef75a,556ef5548b49b2fa22f88a921feae370
4,4,c4ca4238a0b923820dcc509a6f75849b,c81e728d9d4c2f636f067f89cc14862c,f18a6d1cde4b205199de8729a6637b42,c4b022af0f3248bfc492f5b582d81ae8
...,...,...,...,...,...
123367,123367,c81e728d9d4c2f636f067f89cc14862c,e4da3b7fbbce2345d7772b0674a318d5,b7046757c3682a28c5bf2024e57678a0,797ac7c94950128f8bc1e86d9e60fc86
123368,123368,c81e728d9d4c2f636f067f89cc14862c,e4da3b7fbbce2345d7772b0674a318d5,84e8ce7870f0eecd843366582bb95a28,7b597af3514068781199e045f7d38d14
123369,123369,c4ca4238a0b923820dcc509a6f75849b,c4ca4238a0b923820dcc509a6f75849b,c5d9256689c43036581f781c61f26e50,bb82609bac8abe255217a273127fc117
123370,123370,c81e728d9d4c2f636f067f89cc14862c,e4da3b7fbbce2345d7772b0674a318d5,94130ea17023c4837f0dcdda95034b65,576f0dfb3b67052b824172b4fda0695c


In [17]:
class_dict = dict(zip(hierarchy['class'].values, range(len(hierarchy))))
_class = hash_dict_to_df(class_dict, "class")
_class.to_csv("hierarchy_class.csv", index=None)

In [18]:
hierarchy['class'].nunique()

2

In [19]:
_class

,index,class
0,c4ca4238a0b923820dcc509a6f75849b,123369
1,c81e728d9d4c2f636f067f89cc14862c,123371


In [20]:
group_dict = dict(zip(hierarchy['group'].values, range(len(hierarchy))))
_group = hash_dict_to_df(group_dict, "group")
_group.to_csv("hierarchy_group.csv", index=None)

In [25]:
hierarchy['group'].nunique()

5

In [21]:
_group

,index,group
0,c4ca4238a0b923820dcc509a6f75849b,123369
1,e4da3b7fbbce2345d7772b0674a318d5,123371
2,c81e728d9d4c2f636f067f89cc14862c,123366
3,eccbc87e4b5ce2fe28308fd9f2a7baf3,123300
4,1679091c5a880faf6fb5e6087eb1b2dc,122493


In [22]:
category_dict = dict(zip(hierarchy['category'].values, range(len(hierarchy))))
_category = hash_dict_to_df(category_dict, "category")
_category.to_csv("hierarchy_category.csv", index=None)

In [23]:
_category

,index,category
0,7fcc48d22804dbbe9b66b607d51389d4,122673
1,2a3d6d6cc4b5e77238c1fc1bb6cdd681,122608
2,19485224d128528da1602ca47383f078,122367
3,7bec7e63a493e2d61891b1e4051ef75a,123315
4,f18a6d1cde4b205199de8729a6637b42,121283
...,...,...
399,a591024321c5e2bdbd23ed35f0574dde,102649
400,569ff987c643b4bedf504efda8f786c2,112495
401,14f2ebeab937ca128186e7ba876faef9,113664
402,c850c535b6b72487b20cee5d7434506d,78155


In [26]:
hierarchy['category'].nunique()

404

In [28]:
subcategory_dict = dict(zip(hierarchy['subcategory'].values, range(len(hierarchy))))
_subcategory = hash_dict_to_df(subcategory_dict, "subcategory")
_subcategory.to_csv("hierarchy_subcategory.csv", index=None)

In [29]:
hierarchy['subcategory'].nunique()

2120

In [30]:
_subcategory

,index,subcategory
0,7b82e428cf5bdd7671ab0f1fa285a767,121845
1,ec972afde3cb8ff3e84ab57ffe32f8e4,122432
2,34e14de7a374ed14185fa8e81a5ebb1f,102400
3,556ef5548b49b2fa22f88a921feae370,115368
4,c4b022af0f3248bfc492f5b582d81ae8,121283
...,...,...
2115,c5053a380841ebc55d8a0e9f64ef1dd0,107857
2116,81beb8ec9b9d371ce870a11f98fd6e5f,108129
2117,e5f24ee05ff3e224b1487bbdaf704eb2,110014
2118,f699a8a5dcb86ae1165eea7150e36f7a,113511


In [31]:
hierarchy['class'] = hierarchy["class"].map(class_dict)
hierarchy['group'] = hierarchy["group"].map(group_dict)
hierarchy['category'] = hierarchy["category"].map(category_dict)
hierarchy['subcategory'] = hierarchy["subcategory"].map(subcategory_dict)

In [32]:
hierarchy.isna().sum()

sku            0
class          0
group          0
category       0
subcategory    0
dtype: int64

In [33]:
hierarchy.to_csv("hierarchy_preprocessed.csv", index=False)

In [35]:
train.shape

(21692960, 11)

In [36]:
train = train.merge(hierarchy, on='sku', how='left')

In [37]:
train.head()

,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,discount,class,group,category,subcategory
0,2019-10-17,55,20990487,14:35:26,57189,51588,4,1.000,24.94,48.09,2.60,123369,123366,123242,119582
1,2019-10-17,55,20990487,14:35:26,79182,51588,1,1.000,1.14,3.79,0.20,123371,123371,110793,79182
2,2019-10-17,55,20990487,14:35:26,0,51588,8,1.000,85.18,198.18,10.61,123369,123369,122673,121845
3,2019-10-17,55,20990487,14:35:26,71883,51588,3,1.000,29.17,99.99,5.30,123371,123371,123367,123367
4,2019-10-17,55,20990487,14:35:26,26392,171211,5,1.476,157.66,174.79,58.26,123369,123366,122680,121031


In [38]:
train.isna().sum()

day               0
shop_id           0
check_id          0
time              0
sku               0
promo_id          0
check_pos         0
num_sales         0
supplier_price    0
selling_price     0
discount          0
class             0
group             0
category          0
subcategory       0
dtype: int64

In [39]:
shops.isna().sum()

division_id       0
region_id         0
city_id           0
shop_id           0
type_loc_id       0
type_size_id      0
type_format_id    0
type_wealth_id    0
is_store          0
is_active         0
is_ex_billa       0
dtype: int64

In [40]:
shops["shop_id"] = shops["shop_id"].map(shop_id)

In [41]:
division_id_dict = dict(zip(shops['division_id'].values, range(len(shops['division_id']))))
division_id = hash_dict_to_df(division_id_dict, "division_id")
division_id.to_csv("shops_division_id.csv", index=None)

In [42]:
shops['division_id'].nunique()

1

In [43]:
division_id

,index,division_id
0,e4da3b7fbbce2345d7772b0674a318d5,59


In [44]:
region_id_dict = dict(zip(shops['region_id'].values, range(len(shops['division_id']))))
region_id = hash_dict_to_df(region_id_dict, "region_id")
region_id.to_csv("shops_region_id.csv", index=None)

In [45]:
shops['region_id'].nunique()

18

In [47]:
region_id

,index,region_id
0,c0c7c76d30bd3dcaefc96f40275bdc0a,57
1,28dd2c7955ce926456240b2ff0100bde,58
2,9a1158154dfa42caddbd0694a4e9bdc8,37
3,a5bfc9e07964f8dddeb95fc584cd965d,51
4,642e92efb79421734881b53e1e1b18b6,18
5,fbd7939d674997cdb4692d34de8633c4,56
6,d645920e395fedad7bbbed0eca3fe2e0,49
7,f7177163c833dff4b38fc8d2872f1ec6,22
8,d9d4f495e875a2e075a1a4a6e1b9770f,23
9,44f683a84163b3523afe57c2e008bc8c,59


In [48]:
city_id_dict = dict(zip(shops['city_id'].values, range(len(shops['division_id']))))
city_id = hash_dict_to_df(city_id_dict, "city_id")
city_id.to_csv("shops_city_id.csv", index=None)

In [49]:
city_id

,index,city_id
0,3806734b256c27e41ec2c6bffa26d9e7,0
1,1587965fb4d4b5afe8428a4a024feb0d,58
2,204904e461002b28511d5880e1c36a0f,37
3,0768281a05da9f27df178b5c39a51263,51
4,298923c8190045e91288b430794814c4,18
5,aa68c75c4a77c87f97fb686b2f068676,56
6,ce5140df15d046a66883807d18d0264b,49
7,021bbc7ee20b71134d53e20206bd6feb,22
8,08fe2621d8e716b02ec0da35256a998d,23
9,b8c37e33defde51cf91e1e03e51657da,59


In [50]:
shops["division_id"] = shops["division_id"].map(division_id_dict)
shops["region_id"] = shops["region_id"].map(region_id_dict)
shops["city_id"] = shops["city_id"].map(city_id_dict)

In [51]:
shops.isna().sum()

division_id       0
region_id         0
city_id           0
shop_id           0
type_loc_id       0
type_size_id      0
type_format_id    0
type_wealth_id    0
is_store          0
is_active         0
is_ex_billa       0
dtype: int64

In [52]:
shops.to_csv("shops_preprocessed.csv", index=False)

In [54]:
train = train.merge(shops, on="shop_id", how="left")

In [55]:
train.isna().sum()

day               0
shop_id           0
check_id          0
time              0
sku               0
promo_id          0
check_pos         0
num_sales         0
supplier_price    0
selling_price     0
discount          0
class             0
group             0
category          0
subcategory       0
division_id       0
region_id         0
city_id           0
type_loc_id       0
type_size_id      0
type_format_id    0
type_wealth_id    0
is_store          0
is_active         0
is_ex_billa       0
dtype: int64

In [56]:
train.shape

(21692960, 25)

In [58]:
train.to_csv("train_preprocessed_w_features.csv", index=False)

In [59]:
test = pd.read_csv("test_preprocessed.csv")

In [62]:
test.shape

(2666084, 23)

In [61]:
test = test.merge(hierarchy, on='sku', how='left')
test = test.merge(shops, on="shop_id", how="left")

In [63]:
test.isna().sum()

day               0
shop_id           0
check_id          0
time              0
sku               0
promo_id          0
supplier_price    0
selling_price     0
discount          0
class             0
group             0
category          0
subcategory       0
division_id       0
region_id         0
city_id           0
type_loc_id       0
type_size_id      0
type_format_id    0
type_wealth_id    0
is_store          0
is_active         0
is_ex_billa       0
dtype: int64

In [65]:
test.to_csv("test_preprocessed_w_features.csv", index=False)

## Prices for test

In [ ]:
test = pd.read_csv('test_preprocessed_w_features.csv')
test_pr = pd.read_csv('pricing_hackathon_prices_test.csv', names=['day', 'shop_id', 'sku', 'promo_id', 'reg_price', 'promo_price', 'reg_card_price'])

In [ ]:
sku = pd.read_csv('sku.csv', index_col=0).to_dict()['sku']
shop = pd.read_csv('shop_id.csv', index_col=0).to_dict()['shop_id']
promo = pd.read_csv('promo_id.csv', index_col=0).to_dict()['promo_id']
test_pr['shop_id'] = test_pr['shop_id'].map(shop)
test_pr['sku']= test_pr['sku'].map(sku)
test_pr['promo_id'] = test_pr['promo_id'].map(promo)

In [ ]:
test_pr['d'] = test_pr['day'].astype(str).apply(lambda x: str(f'{x[:4]}-{x[4:6]}-{x[6:]}'))
test_pr['d'] = pd.to_datetime(test_pr['d'], format="%Y-%m-%d")
test['day'] = pd.to_datetime(test['day'], format="%Y-%m-%d")
test_pr['day'] = test_pr['d'].copy()

In [ ]:
test_pr = test_pr[test_pr.reg_price>0]
test = test.merge(test_pr[['day', 'shop_id', 'sku', 'promo_id', 'reg_price','promo_price','reg_card_price']],
                 how='left',
                 on = ['day', 'shop_id', 'sku'])

In [ ]:
test['num'] = np.round(((test['selling_price']+test['discount'])/test['reg_price']), 0)
test['price'] = test['selling_price']/test['num']
test['sell_in'] = test['supplier_price']/test['num']
test.loc[test['sell_in'] == np.inf, 'num'] = 1
test['price'] = test['selling_price']/test['num']
test['sell_in'] = test['supplier_price']/test['num']
test.loc[test.sell_in.isna(), 'num'] = 1
test['price'] = test['selling_price']/test['num']
test['sell_in'] = test['supplier_price']/test['num']
test.rename({'promo_id_x': 'promo_id'}, axis=1, inplace=True)
test.drop(columns=['promo_id_y', 'reg_price', 'promo_price', 'reg_card_price'], inplace=True)

In [ ]:
test.to_csv('test_new.csv')